In [1]:
import pickle
import sys
import json
import matplotlib.pyplot as plt
import numpy as np
from collections import defaultdict
import os
import cPickle
%matplotlib inline

In [66]:
items_set=pickle.load(open('processed_data/items_set.pkl','rb')) 
bundle_item_map=pickle.load(open('processed_data/bundle_item_map.pkl','rb')) 
user_bundle_map=pickle.load(open('processed_data/user_bundle_map.pkl','rb')) 
user_item_map=pickle.load(open('processed_data/user_item_map.pkl','rb'))


In [104]:
bundle_discount_percentage_map=pickle.load(open('processed_data/bundle_discount_percentage_map.pkl','rb'))

In [105]:
bundle_discount_array = [bundle_discount_percentage_map[i] for i in range(len(bundle_discount_percentage_map))]

In [4]:
user_item=json.load(open('../../australia_users_items.json'))

In [5]:
user_id_lookup = pickle.load(open('processed_data/user_id_lookup.pkl','rb'))
item_id_lookup = pickle.load(open('processed_data/item_id_lookup.pkl','rb'))


In [6]:
id_user_lookup = dict()
id_item_lookup = dict()
for i in range(len(user_id_lookup)):
    id_user_lookup[user_id_lookup[i]] = i
for i in range(len(item_id_lookup)):
    id_item_lookup[item_id_lookup[i]] = i
    

In [7]:
user_item_played = np.zeros((len(user_item_map.keys()),len(items_set)))
for lines in user_item:
    user_id = lines['user_id']
    if user_id in id_user_lookup:
        user_index = id_user_lookup[user_id]
        for items in lines['items']:
            item_id = int(items['item_id'])
            if item_id in id_item_lookup:
                item_index = id_item_lookup[item_id]
                if item_index in items_set:
                    playtime = items['playtime_forever']
                    user_item_played[user_index][item_index] = int(playtime)       

In [8]:
user_total_playtime = user_item_played.sum(axis=1)
user_item_played_percentage = np.zeros((len(user_item_map.keys()),len(items_set)),dtype=np.float)
for i in range(len(user_item_map.keys())):
    if user_total_playtime[i] != 0:
        for j in range(len(items_set)):
            user_item_played_percentage[i][j] = float(user_item_played[i][j]) / user_total_playtime[i]

In [9]:
item_data=pickle.load(open('processed_data/all_item_data.pkl','rb'))

In [10]:
item_data_map=dict() #appid, item
tags_set=set()
for item in item_data:
    item_data_map[int(item['item_id'])]=item
    for tag in item['genre']:
        tags_set.add(tag)
tags_map=dict()
for i,tag in enumerate(tags_set):
    tags_map[tag]=i 
    
def get_feat(tags):
    feat=np.zeros(len(tags_map))
    for tag in tags:
        feat[tags_map[tag]]=1
    return feat
    
def get_item_data_map_id(index):
    id = item_id_lookup[index]
    for i in range(0,len(item_data)-1):
        if item_data[i]['item_id'] == id:
            return i
            
sgd_tags = np.array([get_feat(item_data[get_item_data_map_id(i)]['genre']) for i in items_set])
print(sgd_tags.shape)

(2515, 327)


In [11]:
user_genre_played_percentage = np.dot(user_item_played_percentage,sgd_tags)
cPickle.dump(user_genre_played_percentage, open('processed_data/user_genre_played_percentage.pkl','w'))

In [ ]:
user_genre_played_percentage = pickle.load(open('processed_data/user_genre_played_percentage.pkl','rb'))

In [12]:
all_data=[] #user,bundle pairs

for user,bundles in user_bundle_map.items():
    for bundle in bundles:
        all_data.append((user,bundle))
        
all_item_data=[]
for user,items in user_item_map.items():
    for item in items:
        all_item_data.append((user,item))

In [13]:
import random
random.shuffle(all_data)
random.shuffle(all_item_data)

In [14]:
cPickle.dump(all_data, open('all_data.pkl','w'))
cPickle.dump(all_item_data,open('all_item_data.pkl','w'))

In [ ]:
all_data = cPickle.load(open('all_data.pkl'))
all_item_data = cPickle.load(open('all_item_data.pkl'))

In [15]:
data_size=len(all_data)

# Training data for bundle for bpr model
training_data=all_data[:int(0.8*data_size)]
test_data=all_data[int(0.8*data_size):]

# Training data for items for bpr_item model
training_data_2=all_item_data[:int(0.8*len(all_item_data))]
test_data_2=all_item_data[int(0.8*len(all_item_data)):]

In [16]:
#if user1's item/bundle is not in user2's bundle map, and user2's item/bundle is not in user1's
def check_tuple(tuple_1, tuple_2, user_bundle_map):
    return tuple_1[1] not in user_bundle_map[tuple_2[0]] and tuple_2[1] not in user_bundle_map[tuple_1[0]]

#negative items and bundles follow the same degree distribution
def graph_sampling(n_samples, training_data, user_bundle_map):
    sgd_users=[]
    sgd_pos_items, sgd_neg_items = [], []
    i=0
    while n_samples>0:
        if i%100000==0:
            print i
        i+=1
        tuple_1=training_data[np.random.randint(len(training_data))]
        tuple_2=training_data[np.random.randint(len(training_data))]
        iteration=100
        while not check_tuple(tuple_1, tuple_2, user_bundle_map):
            tuple_2=training_data[np.random.randint(len(training_data))]
            iteration-=1
            if iteration == 0:
                break
        if iteration==0:
            continue   
        sgd_neg_items.append(tuple_2[1])
        sgd_pos_items.append(tuple_1[1])
        sgd_users.append(tuple_1[0])
        
        sgd_neg_items.append(tuple_1[1])
        sgd_pos_items.append(tuple_2[1])
        sgd_users.append(tuple_2[0])
        n_samples-=2
    return sgd_users, sgd_pos_items, sgd_neg_items

In [17]:
# Generting training data for items through graph sampling.
sgd_train_users_items, sgd_train_pos_items, sgd_train_neg_items = graph_sampling(len(training_data_2)*30, training_data_2, user_item_map)

0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
2800000
2900000
3000000
3100000
3200000
3300000
3400000
3500000
3600000
3700000
3800000
3900000
4000000
4100000
4200000
4300000
4400000
4500000
4600000
4700000
4800000
4900000
5000000
5100000
5200000
5300000
5400000
5500000
5600000
5700000
5800000
5900000
6000000
6100000
6200000
6300000
6400000
6500000
6600000
6700000
6800000
6900000
7000000
7100000
7200000
7300000
7400000
7500000
7600000
7700000
7800000
7900000
8000000
8100000
8200000
8300000
8400000
8500000
8600000
8700000
8800000
8900000
9000000
9100000
9200000
9300000


In [18]:
cPickle.dump(sgd_train_users_items, open('sgd_train_users_items.pkl','w'))
cPickle.dump(sgd_train_pos_items, open('sgd_train_pos_items.pkl','w'))
cPickle.dump(sgd_train_neg_items, open('sgd_train_neg_items.pkl','w'))

In [19]:
def get_test_data_items(test_data, train_data):
    users=[]
    pos_items=[]
    neg_items=[]
    train_dict, train_users, train_items  = data_to_dict(train_data)
    test_dict, test_users, test_items = data_to_dict(test_data)
    z = 0
    for i,user in enumerate(test_dict.keys()):
        if(i%1000==0):
            print i

        if user in train_users: 
            for pos_item in test_dict[user]:
                if pos_item in train_items:
                    for neg_item in train_items:
                        if neg_item not in test_dict[user] and neg_item not in train_dict[user]:
                            users.append(user)
                            pos_items.append(pos_item)
                            neg_items.append(neg_item)

    return users, pos_items, neg_items


def data_to_dict(data):
    data_dict = defaultdict(list)
    items = set()
    for (user, item) in data:
        data_dict[user].append(item)
        items.add(item)
    return data_dict, set(data_dict.keys()), items

In [20]:
test_users_cold, test_pos_items_cold, test_neg_items_cold = get_test_data_items(test_data_2, training_data_2)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000


In [ ]:
cPickle.dump(test_users_cold, open('test_users_cold.pkl','w'))
cPickle.dump(test_pos_items_cold, open('test_pos_items_cold.pkl','w'))
cPickle.dump(test_neg_items_cold, open('test_neg_items_cold.pkl','w'))

In [ ]:
sgd_train_users_items = cPickle.load(open('sgd_train_users_items.pkl'))
sgd_train_pos_items = cPickle.load(open('sgd_train_pos_items.pkl'))
sgd_train_neg_items = cPickle.load(open('sgd_train_neg_items.pkl'))

In [33]:
new_test_users=[]
new_test_pos=[]
new_test_neg=[]

for i in range(len(test_users_cold)):
    if(i%45==0):
        new_test_users.append(test_users_cold[i])
        new_test_pos.append(test_pos_items_cold[i])
        new_test_neg.append(test_neg_items_cold[i])

In [35]:
test_users_cold,test_pos_items_cold,test_neg_items_cold = new_test_users, new_test_pos, new_test_neg

In [34]:
len(new_test_users)

4018576

In [36]:
cPickle.dump(test_users_cold, open('test_users_cold.pkl','w'))
cPickle.dump(test_pos_items_cold, open('test_pos_items_cold.pkl','w'))
cPickle.dump(test_neg_items_cold, open('test_neg_items_cold.pkl','w'))

In [ ]:
test_users_cold = cPickle.load(open('test_users_cold.pkl'))
test_pos_items_cold = cPickle.load(open('test_pos_items_cold.pkl'))
test_neg_items_cold = cPickle.load(open('test_neg_items_cold.pkl'))

In [23]:
print len(sgd_train_users_items)

18615300


In [37]:
import os
os.environ['THEANO_FLAGS']='mode=FAST_RUN,device=gpu,lib.cnmem=0.7,floatX=float32'

In [94]:
# theano-bpr
#
# Copyright (c) 2014 British Broadcasting Corporation
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import theano, numpy
import theano.tensor as T
import time
import sys
from collections import defaultdict

class BPR_Item(object):

    def __init__(self, rank, n_users, n_items, n_tags, lambda_u = 0.0025, lambda_i = 0.0025, lambda_j = 0.00025, lambda_d = 0.00025, lambda_p = 0.00025, lambda_q = 0.00025, lambda_bias = 0.0, learning_rate = 0.05):
        
        self._rank = rank
        self._n_users = n_users
        self._n_items = n_items
        self._n_tags = n_tags
        self._lambda_u = lambda_u
        self._lambda_i = lambda_i
        self._lambda_j = lambda_j
        self._lambda_d = lambda_d
        self._lambda_p = lambda_p
        self._lambda_q = lambda_q
        self._lambda_bias = lambda_bias
        self.A = theano.shared(sgd_tags.astype("float32"), name='A')
        #self.G = theano.shared(user_genre_played_percentage.astype("float32"), name='G')#user x genre
        self._learning_rate = learning_rate
        self._configure_theano()
        self._generate_train_model_function()
        self._generate_test_model_function()
        

    def _configure_theano(self):
        """
          Configures Theano to run in fast mode
          and using 32-bit floats. 
        """
        theano.config.mode = 'FAST_RUN'
        theano.config.floatX = 'float32'

    def _generate_train_model_function(self):
        u = T.lvector('u')
        i = T.lvector('i')
        j = T.lvector('j')

        h = numpy.random.random((self._n_items, self._rank))
        b = numpy.random.random(self._n_items)#1*n_items
        x = numpy.random.random((self._n_users,self._n_tags))

        #### needs change ###
        
        self.W = theano.shared(numpy.random.random((self._n_users, self._rank)).astype('float32'), name='W')#P
        self.H = theano.shared(h.astype('float32'), name='H')#Q
        self.X = theano.shared(x.astype('float32'), name='X')
        self.B = theano.shared(b.astype('float32'), name='B')#beta
    
        #x_ui = B_i+P_u*Q_i+X_u*A_i
        x_ui = T.dot(self.W[u], self.H[i].T).diagonal() + self.B[i] + T.dot(self.X[u], self.A[i].T).diagonal()
        x_uj = T.dot(self.W[u], self.H[j].T).diagonal() + self.B[j] + T.dot(self.X[u], self.A[j].T).diagonal()

        x_uij = T.nnet.sigmoid(x_ui - x_uj)
        
        obj = T.sum(T.log(x_uij) - self._lambda_u * (self.W[u] ** 2).sum(axis=1) - 
                    self._lambda_i * (self.H[i] ** 2).sum(axis=1) - self._lambda_j * 
                    (self.H[j] ** 2).sum(axis=1) - self._lambda_bias * 
                    (self.B[i] ** 2 + self.B[j] ** 2) - self._lambda_d * (self.X[u] ** 2).sum(axis=1))
        
        cost = - obj

        g_cost_W = T.grad(cost=cost, wrt=self.W)
        g_cost_H = T.grad(cost=cost, wrt=self.H)
        g_cost_B = T.grad(cost=cost, wrt=self.B)
        g_cost_X = T.grad(cost=cost, wrt=self.X)

        updates = [(self.W, self.W - self._learning_rate * g_cost_W), (self.H, self.H - self._learning_rate * g_cost_H), 
                   (self.B, self.B - self._learning_rate * g_cost_B), (self.X, self.X - self._learning_rate * g_cost_X)]
        
                                                  #u, i_p, i_n    
        self.train_model = theano.function(inputs=[u, i, j], outputs=cost, updates=updates)

    
    def train(self, s_users=None, s_pos_items=None, s_neg_items=None, batch_size=1000):
        """
          Trains the BPR Matrix Factorisation model using Stochastic
          Gradient Descent and minibatches over `train_data`.

          `train_data` is an array of (user_index, item_index) tuples.

          We first create a set of random samples from `train_data` for 
          training, of size `epochs` * size of `train_data`.

          We then iterate through the resulting training samples by
          batches of length `batch_size`, and run one iteration of gradient
          descent for the batch.
        """
        if len(s_pos_items) < batch_size:
            sys.stderr.write("WARNING: Batch size is greater than number of training samples, switching to a batch size of %s\n" % str(len(train_data)))
            batch_size = len(s_pos_items)
            
        sgd_users, sgd_pos_items, sgd_neg_items = s_users, s_pos_items, s_neg_items
        n_sgd_samples = len(s_users)
        
        z = 0
        t2 = t1 = t0 = time.time()
        while (z+1)*batch_size < n_sgd_samples:
            self.train_model(
                sgd_users[z*batch_size: (z+1)*batch_size],
                sgd_pos_items[z*batch_size: (z+1)*batch_size],
                sgd_neg_items[z*batch_size: (z+1)*batch_size]
            )
            z += 1
            t2 = time.time()
            sys.stderr.write("\rProcessed %s ( %.2f%% ) in %.4f seconds" %(str(z*batch_size), 100.0 * float(z*batch_size)/n_sgd_samples, t2 - t1))
            sys.stderr.flush()
            t1 = t2
        if n_sgd_samples > 0:
            sys.stderr.write("\nTotal training time %.2f seconds; %e per sample\n" % (t2 - t0, (t2 - t0)/n_sgd_samples))
            sys.stderr.flush()
            
    def _generate_test_model_function(self):
        """
          Computes item predictions for `user_index`.
          Returns an array of prediction values for each item
          in the dataset.
        """
        u = T.lvector('u')
        i = T.lvector('i')
        j = T.lvector('j')

        x_ui = T.dot(self.W[u], self.H[i].T).diagonal() + self.B[i] + T.dot(self.X[u], self.A[i].T).diagonal()
        x_uj = T.dot(self.W[u], self.H[j].T).diagonal() + self.B[j] + T.dot(self.X[u], self.A[j].T).diagonal()
        x_uij = x_ui - x_uj
        
        self.test_model = theano.function(inputs=[u, i, j], outputs=x_uij)
   
    def test_bundle(self, sgd_users, sgd_pos_items, sgd_neg_items, batch_size=1000):
        """
          Computes the Area Under Curve (AUC) on `test_data`.

          `test_data` is an array of (user_index, item_index) tuples.

          During this computation we ignore users and items
          that didn't appear in the training data, to allow
          for non-overlapping training and testing sets.
        """
        
        auc_values = []
        z = 0
        t2 = t1 = t0 = time.time()
        n_sgd_samples = len(sgd_users)
        while (z+1)*batch_size < n_sgd_samples:
            pref_list=self.test_model(
                sgd_users[z*batch_size: (z+1)*batch_size],
                sgd_pos_items[z*batch_size: (z+1)*batch_size],
                sgd_neg_items[z*batch_size: (z+1)*batch_size]
            )
            z += 1
            t2 = time.time()
            sys.stderr.write("\rProcessed %s ( %.2f%% ) in %.4f seconds" %(str(z*batch_size), 100.0 * float(z*batch_size)/n_sgd_samples, t2 - t1))
            t1 = t2
            
            auc = np.sum([1.0 if a>0.0 else 0.0 for a in pref_list])
            auc /= batch_size
            
            auc_values.append(auc)
            sys.stderr.write("\rCurrent AUC mean (%s samples): %0.5f" % (str(z*batch_size), numpy.mean(auc_values)))
            sys.stderr.flush()
        
        sys.stderr.write("\n")
        sys.stderr.flush()
        return numpy.mean(auc_values)

In [95]:
bpr_item = BPR_Item(10, len(user_item_map.keys()), len(items_set), len(tags_map))


In [96]:
bpr_item.train(s_users=sgd_train_users_items, s_pos_items=sgd_train_pos_items, s_neg_items=sgd_train_neg_items)

Processed 18615000 ( 100.00% ) in 0.0728 seconds
Total training time 1324.33 seconds; 7.114187e-05 per sample


In [97]:
bpr_item.test_bundle(test_users_cold, test_pos_items_cold, test_neg_items_cold)

Current AUC mean (4018000 samples): 0.88832nds


0.88832105525136895

In [87]:
cPickle.dump(bpr_item, open('bpr_item.pkl','w'))

In [ ]:
bpr_item = cPickle.load(open('bpr_item.pkl'))

# Bundle model begins

In [46]:
# Generting training data for bundles through graph sampling.
sgd_users, sgd_pos_bundles, sgd_neg_bundles = graph_sampling(len(training_data)*30, training_data, user_bundle_map)


0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000


In [47]:
cPickle.dump(sgd_users, open('sgd_users.pkl', 'w'))
cPickle.dump(sgd_pos_bundles, open('sgd_pos_bundles.pkl', 'w'))
cPickle.dump(sgd_neg_bundles, open('sgd_neg_bundles.pkl', 'w'))

In [67]:
# Determining max bundle size to create bins for N
max_bundle_size=0
for bundle,items in bundle_item_map.items():
    if(len(items)>max_bundle_size):
        max_bundle_size=len(items)
print max_bundle_size
print len(items_set)

82
2515


In [68]:
def get_items(bundle_id, max_bundle_size, index):
    item=list(bundle_item_map[bundle_id]);
    for i in range(len(item),max_bundle_size):
        item.append(index)
    return item

sgd_pos_items=[get_items(b_id, max_bundle_size, len(items_set)) for b_id in sgd_pos_bundles]
sgd_neg_items=[get_items(b_id, max_bundle_size, len(items_set)) for b_id in sgd_neg_bundles]


In [86]:
cPickle.dump(sgd_pos_items, open('sgd_pos_items.pkl', 'w'))
cPickle.dump(sgd_neg_items, open('sgd_neg_items.pkl', 'w'))

In [69]:
from sklearn.metrics import jaccard_similarity_score
from sklearn.metrics.pairwise import cosine_similarity
def compute_diversity_tags(app_data):
    l=len(app_data)
    app_data=[item_id_lookup[d] for d in app_data]
    count=0.0
    similarity=0.0
    for i in range(l):
        if app_data[i] in item_data_map:
            for j in range(i+1,l):
                if app_data[j] in item_data_map:
                    count+=1
                    similarity+=jaccard_similarity_score(get_feat(item_data_map[app_data[i]]['tags']),
                                                         get_feat(item_data_map[app_data[j]]['tags']))
    if count>0:
        return 1-(similarity/count)
    else:
        return 0.0

def compute_diversity_latent(app_data, H):
    l=len(app_data)
    count=0.0
    similarity=0.0
    for i in range(l):
            for j in range(i+1,l):
                itemID1,itemID2 = app_data[i],app_data[j]
                count+=1
                similarity+=cosine_similarity(H[itemID1].reshape(1,-1),H[itemID2].reshape(1,-1))[0,0]
    if count>0:
        return 1-(similarity/count)
    else:
        return 0.0
    
def compute_diversity(app_data, H):
    if H is not None:
        return compute_diversity_latent(app_data, H)
    else :
        return compute_diversity_tags(app_data)

In [70]:
Gamma=bpr_item.H.eval()
bundle_diversity_map=dict()
for bundle,items in bundle_item_map.items():
    bundle_diversity_map[bundle]=compute_diversity_latent(list(items), bpr_item.H.eval())

#bundle_diversity_map=pickle.load(open('../../data/pickle/training_data/game_aus/bpr/bundle_diversity_map','rb'))

In [71]:
sgd_pos_len=[len(bundle_item_map[b_id]) for b_id in sgd_pos_bundles]
sgd_neg_len=[len(bundle_item_map[b_id]) for b_id in sgd_neg_bundles]
sgd_pos_diversity=[bundle_diversity_map[b_id] for b_id in sgd_pos_bundles]
sgd_neg_diversity=[bundle_diversity_map[b_id] for b_id in sgd_neg_bundles]

In [73]:
def get_test_data_bundles(test_data, train_data, n_items):
    users=[]
    pos_items=[]
    neg_items=[]
    n1=[]
    n2=[]
    pos_diversity=[]
    neg_diversity=[]
    train_dict, train_users, train_items  = data_to_dict(train_data)
    test_dict, test_users, test_items = data_to_dict(test_data)
    auc_values = []
    z = 0
    for i,user in enumerate(test_dict.keys()):
        if(i%1000==0):
            print i

        if user in train_users: 
            for pos_item in test_dict[user]:
                if pos_item in train_items:
                    for neg_item in train_items:
                        if neg_item not in test_dict[user] and neg_item not in train_dict[user]:
                            pos_diversity.append(bundle_diversity_map[pos_item])
                            neg_diversity.append(bundle_diversity_map[neg_item])
                            users.append(user)
                            pos_items.append(pos_item)
                            neg_items.append(neg_item)
                            n1.append(len(bundle_item_map[pos_item]))
                            n2.append(len(bundle_item_map[neg_item]))

    pos_items=[get_items(b_id, max_bundle_size, n_items) for b_id in pos_items]
    neg_items=[get_items(b_id, max_bundle_size, n_items) for b_id in neg_items]
    return users, pos_items, neg_items, n1, n2, pos_diversity, neg_diversity


In [74]:
test_users, test_pos_items, test_neg_items, test_n1, test_n2, test_pos_diversity, test_neg_diversity= get_test_data_bundles(test_data, training_data, len(items_set))

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000


In [80]:
cPickle.dump(test_users, open('test_users.pkl', 'w'))
cPickle.dump(test_pos_items, open('test_pos_items.pkl', 'w'))
cPickle.dump(test_neg_items, open('test_neg_items.pkl', 'w'))
cPickle.dump(test_n1, open('test_n1.pkl', 'w'))
cPickle.dump(test_n2, open('test_n2.pkl', 'w'))
cPickle.dump(test_pos_diversity, open('test_pos_diversity.pkl', 'w'))
cPickle.dump(test_neg_diversity, open('test_neg_diversity.pkl', 'w'))
cPickle.dump(sgd_pos_len, open('sgd_pos_len.pkl', 'w'))
cPickle.dump(sgd_neg_len, open('sgd_neg_len.pkl', 'w'))
cPickle.dump(sgd_pos_diversity, open('sgd_pos_diversity.pkl', 'w'))
cPickle.dump(sgd_neg_diversity, open('sgd_neg_diversity.pkl', 'w'))

In [82]:
cPickle.dump(bundle_diversity_map, open('bundle_diversity_map.pkl', 'w'))

In [75]:
print np.shape(bpr_item.H.eval())
H_item=bpr_item.H.eval()
H_item = np.concatenate((H_item,np.zeros((1,np.shape(H_item)[1]))),axis=0)
H_item=np.array(H_item).astype('float32')
print np.shape(H_item)

print np.shape(bpr_item.B.eval())
B_item=bpr_item.B.eval()
B_item = np.append(B_item,0)
B_item=np.array(B_item).astype('float32')
print np.shape(B_item)

print np.shape(bpr_item.A.eval())
A_item=bpr_item.A.eval()
A_item = np.concatenate((A_item,np.zeros((1,np.shape(A_item)[1]))),axis=0)
A_item=np.array(A_item).astype('float32')
print np.shape(A_item)

(2515, 10)
(2516, 10)
(2515,)
(2516,)
(2515, 327)
(2516, 327)


In [118]:
import theano, numpy
import theano.tensor as T
import time
import sys
from collections import defaultdict

class BPR_Cold(object):

    def __init__(self, rank, bundle_size, n_users, n_items, n_tags, lambda_u = 0.0025, lambda_i = 0.01, lambda_j = 0.0025, lambda_d = 0.0025, lambda_p = 0.00025, lambda_bias = 0.0, learning_rate = 0.05):
        
        self._rank = rank
        self._bundle_rank = bundle_size + 1 #only for penalty
        self._n_users = n_users
        self._n_items = n_items
        self._n_tags = n_tags
        self._lambda_u = lambda_u
        self._lambda_i = lambda_i
        self._lambda_j = lambda_j
        self._lambda_d = lambda_d
        self._lambda_p = lambda_p
        self._lambda_bias = lambda_bias
        self._learning_rate = learning_rate
        self.bundle_discount = theano.shared(np.array(bundle_discount_array).astype('float32'), name='bundle_discount')
        self._configure_theano()
        self._generate_train_model_item_function()
        self._generate_test_model_function()

    def _configure_theano(self):
        theano.config.mode = 'FAST_RUN'
        theano.config.floatX = 'float32'
    
    def _generate_train_model_item_function(self):
        u = T.lvector('u')
        i = T.lmatrix('i')
        j = T.lmatrix('j')
        n1 = T.lvector('n1')#num of items in pos bundle
        n2 = T.lvector('n2')
        #di = T.dvector('di')#cb
        #dj = T.dvector('dj')#cb
        
        pu = numpy.random.random(self._n_users)
        
        self.W1 = bpr_item.W #Pu
        self.H1 = theano.shared(H_item.astype('float32'), name='H')#Qi
        self.B1 = theano.shared(B_item.astype('float32'), name='B')#Bi
        self.X = bpr_item.X
        self.A = theano.shared(A_item.astype('float32'), name='A')#Ai
        #self.Pu = theano.shared(pu.astype('float32'), name='Pu')
        
        self.M1 = theano.shared(numpy.random.random((self._rank, self._rank)).astype('float64'), name='M1')#w
        self.M2 = theano.shared(numpy.random.random((self._rank, self._rank)).astype('float64'), name='M2')#u
        self.E1 = theano.shared(numpy.random.random((self._n_tags,self._n_tags)).astype('float64'), name='E1')#theta
        self.E2 = theano.shared(numpy.random.random((self._n_tags,self._n_tags)).astype('float64'), name='E2')#alpha
        self.K = theano.shared(numpy.random.rand(), name='K')#k
        #self.D = theano.shared(numpy.random.rand(), name='D')#C
        self.N = theano.shared(numpy.random.random(self._bundle_rank).astype('float32'), name='N')#Nb
        
        #bi = self.bundle_discount[i]
        #bj = self.bundle_discount[j]
        
        x_ui = T.dot(T.dot(self.W1[u],self.M2), T.dot(self.M1, self.H1[i].sum(axis=1).T/n1)).diagonal() + T.dot(T.dot(self.X[u],self.E1),T.dot(self.E2,self.A[i].sum(axis=1).T/n1)).diagonal() + self.K*(self.B1[i].T/n1).T.sum(axis=1) + self.N[n1] #+ self.D*di
        x_uj = T.dot(T.dot(self.W1[u],self.M2), T.dot(self.M1, self.H1[j].sum(axis=1).T/n2)).diagonal() + T.dot(T.dot(self.X[u],self.E1),T.dot(self.E2,self.A[j].sum(axis=1).T/n2)).diagonal() + self.K*(self.B1[j].T/n2).T.sum(axis=1) + self.N[n2] #+ self.D*dj
        
        x_uij = T.nnet.sigmoid(x_ui-x_uj) #+ (self.Pu[u]) *(bi - bj)
        
        obj = T.sum(T.log(x_uij) - self._lambda_u * (self.M1 ** 2).sum() - \
                    self._lambda_u * (self.M2 ** 2).sum()  - self._lambda_d * (self.K**2) #- self._lambda_j * (self.D**2)\
                    - self._lambda_p * (self.N[n2]**2) - self._lambda_p * (self.N[n1]**2) - self._lambda_i * (self.E1 **2).sum() - \
                   self._lambda_i * (self.E2 ** 2).sum())# - self._lambda_j * (self.Pu ** 2))
        cost = - obj

        g_cost_M1 = T.grad(cost=cost, wrt=self.M1)
        g_cost_M2 = T.grad(cost=cost, wrt=self.M2)
        g_cost_K = T.grad(cost=cost, wrt=self.K)
        g_cost_N = T.grad(cost=cost, wrt=self.N)
        #g_cost_D = T.grad(cost=cost, wrt=self.D)
        g_cost_E1 = T.grad(cost=cost, wrt=self.E1)
        g_cost_E2 = T.grad(cost=cost, wrt=self.E2)
        
        
        updates = [(self.M1, self.M1 - self._learning_rate * .001* g_cost_M1), (self.M2, self.M2 - self._learning_rate *.001* g_cost_M2), 
                   (self.K, self.K - self._learning_rate * .001*g_cost_K), (self.N, self.N - self._learning_rate *g_cost_N),
                  #(self.D, self.D - self._learning_rate * g_cost_D), 
                   (self.E1, self.E1 - self._learning_rate *.001*g_cost_E1),(self.E2, self.E2 - self._learning_rate *.001 *g_cost_E2)]
                  

        self.train_model_item = theano.function(inputs=[u, i, j, n1, n2], outputs=cost, updates=updates)

    
    def train(self, s_users=None, s_pos_items=None, s_neg_items=None, s_pos_len=None, s_neg_len=None,
             s_pos_diversity=None, s_neg_diversity=None,batch_size=1000):
        
        if len(s_users) < batch_size:
            sys.stderr.write("WARNING: Batch size is greater than number of training samples, switching to a batch size of %s\n" % str(len(train_data)))
            batch_size = len(s_users)
        
        sgd_users, sgd_pos_items, sgd_neg_items = s_users, s_pos_items, s_neg_items
        n_sgd_samples = len(s_users)

        z = 0
        t2 = t1 = t0 = time.time()
        while (z+1)*batch_size < n_sgd_samples:
            
            self.train_model_item(
                sgd_users[z*batch_size: (z+1)*batch_size],
                sgd_pos_items[z*batch_size: (z+1)*batch_size],
                sgd_neg_items[z*batch_size: (z+1)*batch_size],
                s_pos_len[z*batch_size: (z+1)*batch_size],
                s_neg_len[z*batch_size: (z+1)*batch_size]
            )
            z += 1
            t2 = time.time()
            sys.stderr.write("\rProcessed %s ( %.2f%% ) in %.4f seconds" %(str(z*batch_size), 100.0 * float(z*batch_size)/n_sgd_samples, t2 - t1))
            sys.stderr.flush()
            t1 = t2
        if n_sgd_samples > 0:
            sys.stderr.write("\nTotal training time %.2f seconds; %e per sample\n" % (t2 - t0, (t2 - t0)/n_sgd_samples))
            sys.stderr.flush()
    
    def _generate_test_model_function(self):
        u = T.lvector('u')
        i = T.lmatrix('i')
        j = T.lmatrix('j')
        n1 = T.lvector('n1')
        n2 = T.lvector('n2')
        #di = T.dvector('di')
        #dj = T.dvector('dj')
        
        x_ui = T.dot(T.dot(self.W1[u],self.M2), T.dot(self.M1, self.H1[i].sum(axis=1).T/n1)).diagonal() + T.dot(T.dot(self.X[u],self.E1),T.dot(self.E2,self.A[i].sum(axis=1).T/n1)).diagonal() + self.K*(self.B1[i].T/n1).T.sum(axis=1) + self.N[n1] #+ self.D*di
        x_uj = T.dot(T.dot(self.W1[u],self.M2), T.dot(self.M1, self.H1[j].sum(axis=1).T/n2)).diagonal() + T.dot(T.dot(self.X[u],self.E1),T.dot(self.E2,self.A[j].sum(axis=1).T/n2)).diagonal() + self.K*(self.B1[j].T/n2).T.sum(axis=1) + self.N[n2] #+ self.D*dj
        #bi = self.bundle_discount[i]
        #bj = self.bundle_discount[j]
        x_uij = x_ui-x_uj #+ (self.Pu[u]) *(bi - bj)
        self.test_model = theano.function(inputs=[u, i, j, n1, n2], outputs=x_uij)
        
    def test_bundle(self, sgd_users, sgd_pos_items, sgd_neg_items, s_pos_len, s_neg_len, s_pos_diversity, s_neg_diversity, batch_size=1000):
        
        auc_values = []
        z = 0
        t2 = t1 = t0 = time.time()
        n_sgd_samples = len(sgd_users)
        while (z+1)*batch_size < n_sgd_samples:
            pref_list=self.test_model(
                sgd_users[z*batch_size: (z+1)*batch_size],
                sgd_pos_items[z*batch_size: (z+1)*batch_size],
                sgd_neg_items[z*batch_size: (z+1)*batch_size],
                s_pos_len[z*batch_size: (z+1)*batch_size],
                s_neg_len[z*batch_size: (z+1)*batch_size]
            )
            z += 1
            t2 = time.time()
            sys.stderr.write("\rProcessed %s ( %.2f%% ) in %.4f seconds" %(str(z*batch_size), 100.0 * float(z*batch_size)/n_sgd_samples, t2 - t1))
            t1 = t2
            
            auc = np.sum([1.0 if a>0.0 else 0.0 for a in pref_list])
            auc /= batch_size
            
            auc_values.append(auc)
            sys.stderr.write("\rCurrent AUC mean (%s samples): %0.5f" % (str(z*batch_size), numpy.mean(auc_values)))
            sys.stderr.flush()
        
        sys.stderr.write("\n")
        sys.stderr.flush()
        return numpy.mean(auc_values)
  

In [119]:
bpr_cold = BPR_Cold(10, max_bundle_size, len(user_bundle_map.keys()), len(items_set), len(tags_map))

In [120]:
bpr_cold.train(s_users=sgd_users, s_pos_items=sgd_pos_items, s_neg_items=sgd_neg_items, 
          s_pos_len=sgd_pos_len, s_neg_len=sgd_neg_len, s_pos_diversity=sgd_pos_diversity, s_neg_diversity=sgd_neg_diversity)
#bpr_cold.train(s_users=sgd_users, s_pos_items=sgd_pos_items, s_neg_items=sgd_neg_items, 
#          s_pos_len=sgd_pos_len, s_neg_len=sgd_neg_len, s_pos_diversity=sgd_pos_diversity, s_neg_diversity=sgd_neg_diversity)

Processed 2101000 ( 99.99% ) in 0.3103 seconds
Total training time 667.56 seconds; 3.177136e-04 per sample


In [121]:
bpr_cold.test_bundle(test_users, test_pos_items, test_neg_items, test_n1, test_n2, test_pos_diversity, test_neg_diversity)

Current AUC mean (1263000 samples): 0.91900nds

KeyboardInterrupt: 

# Bundle Generation Through Greedy Model Begins

In [83]:
def generate_bundle(items_set, user, initial_size = 3, max_iteration = 1000, sample_size = 5):
    current_bundle = np.random.choice(list(items_set), initial_size)
    
    T=1000.0
    
    iteration = 0
    while iteration < max_iteration:
        iteration+=1
        curr_diversity = compute_diversity(current_bundle, Gamma)
        user_set=[]
        pos_item_set=[]
        actual_item_set=[]
        neg_item_set=[]
        pos_item_count=[]
        neg_item_count=[]
        pos_diversity=[]
        neg_diversity=[]
        
        
        candidate_items = set(np.random.choice(list(items_set), sample_size))
        
        for item in current_bundle:
            if item in candidate_items:
                candidate_items.remove(item)
        
    
        #Generating new bundles by adding and removing new items  
        for cand_item in candidate_items:
            #Add an item case
            if len(current_bundle)<10:
                user_set.append(user)    
                neg_item_set.append(add_bogus_items(current_bundle , max_bundle_size, len(items_set)))
                neg_item_count.append(len(current_bundle))
                neg_diversity.append(curr_diversity)         
                new_bundle=list(current_bundle)
                new_bundle.append(cand_item)
                pos_item_count.append(len(new_bundle))
                pos_diversity.append(compute_diversity(new_bundle, Gamma))
                actual_item_set.append(new_bundle)
                pos_item_set.append(add_bogus_items(new_bundle , max_bundle_size, len(items_set)))

            # Replace an item case
            for curr_item in current_bundle:
                user_set.append(user)
                
                neg_item_set.append(add_bogus_items(current_bundle , max_bundle_size, len(items_set)))
                neg_item_count.append(len(current_bundle))
                neg_diversity.append(curr_diversity)
                
                new_bundle=list(current_bundle)
                new_bundle.append(cand_item)
                new_bundle.remove(curr_item)
                pos_item_set.append(add_bogus_items(new_bundle , max_bundle_size, len(items_set)))
                actual_item_set.append(new_bundle)
                pos_item_count.append(len(new_bundle))
                pos_diversity.append(compute_diversity(new_bundle, Gamma))
         
        
        # Remove an item case
        if len(current_bundle)>2:
            for curr_item in current_bundle:
                user_set.append(user)

                neg_item_set.append(add_bogus_items(current_bundle , max_bundle_size, len(items_set)))
                neg_item_count.append(len(current_bundle))
                neg_diversity.append(curr_diversity)

                new_bundle=list(current_bundle)
                new_bundle.remove(curr_item)
                actual_item_set.append(new_bundle)
                pos_item_set.append(add_bogus_items(new_bundle , max_bundle_size, len(items_set)))
                pos_item_count.append(len(new_bundle))
                pos_diversity.append(compute_diversity(new_bundle, Gamma))
        
                
        pref_score = bpr_cold.test_model(user_set, pos_item_set, neg_item_set, pos_item_count, 
                                    neg_item_count, pos_diversity, neg_diversity)
                                    
        #print pref_score, pos_item_count, neg_item_count
        index = np.argmax(pref_score)
        #print "Pref Score ", pref_score[index]
        if(pref_score[index]>0):
            current_bundle = actual_item_set[index]
        else:
            prob = np.exp(pref_score[index]/T)
            if prob < .00001:
                break
            if np.random.rand() < prob:
                current_bundle = actual_item_set[index]
        T=T*0.9
    #print iteration
    return current_bundle 

In [84]:
def add_bogus_items(bundle, max_bundle_size, index):
    item=list(bundle);
    for i in range(len(item),max_bundle_size):
        item.append(index)
    return item

def remove_bogus_items(bundle, max_bundle_size, index):
    item=list(bundle);
    i=0
    while i< len(bundle):
        if bundle[i]==index:
            break
        i+=1
    return bundle[:i]

def get_bundle_rank(user, new_bundle, bundle_item_map, bundle_diversity_map):
    user_set=[]
    pos_item_set=[]
    neg_item_set=[]
    pos_item_count=[]
    neg_item_count=[]
    pos_diversity=[]
    neg_diversity=[]
    
    bundle_diversity=compute_diversity(new_bundle, Gamma)
    for bundle_id,bundle in bundle_item_map.items():
        user_set.append(user)
        pos_item_set.append(add_bogus_items(bundle, max_bundle_size, len(items_set)))
        neg_item_set.append(add_bogus_items(new_bundle, max_bundle_size, len(items_set)))
        pos_item_count.append(len(bundle))
        neg_item_count.append(len(new_bundle))
        pos_diversity.append(bundle_diversity_map[bundle_id])
        neg_diversity.append(bundle_diversity)
        
    pref_score = bpr_cold.test_model(user_set, pos_item_set, neg_item_set, pos_item_count, 
                                    neg_item_count, pos_diversity, neg_diversity)
  
    rank = np.sum([1.0 if p>0 else 0.0 for p in pref_score])
    return rank

In [85]:
sizes=[10]
diversities=[]
scores=[]
bundle_sizes=[]
for size in sizes:
    aggregate_diversity=set()
    pred_score=[]
    b_size=[]
    generated_bundles=[]
    for user in sorted(user_bundle_map.keys())[:100]:
        new_bundle = generate_bundle(items_set, user, 4, 1000,size)
        rank = get_bundle_rank(user, new_bundle, bundle_item_map, bundle_diversity_map)
        purchased_bundles = len(user_bundle_map[user])
        aggregate_diversity=aggregate_diversity.union(set(new_bundle))
        generated_bundles.append(new_bundle)
        pred_score.append(rank)
        b_size.append(len(new_bundle)*1.0)
        print 'Rank of user %d : %d, Size of bundle : %d, Bundles purchased : %d Aggregate diversity: %d Score: %f, Average bundle size: %f' %(user, 
                                                                                     rank, 
                                                                                     len(new_bundle),                                 
                                                                                     purchased_bundles,
                                                                                     len(aggregate_diversity),
                                                                                     1.0+np.mean(pred_score),
                                                                                     np.mean(b_size))
    diversities.append(len(aggregate_diversity))
    scores.append(1.0+np.mean(pred_score))
    bundle_sizes.append(np.mean(b_size)*100)
    

Rank of user 0 : 0, Size of bundle : 2, Bundles purchased : 6 Aggregate diversity: 2 Score: 1.000000, Average bundle size: 2.000000
Rank of user 1 : 1, Size of bundle : 2, Bundles purchased : 15 Aggregate diversity: 4 Score: 1.500000, Average bundle size: 2.000000
Rank of user 2 : 1, Size of bundle : 2, Bundles purchased : 1 Aggregate diversity: 6 Score: 1.666667, Average bundle size: 2.000000
Rank of user 3 : 0, Size of bundle : 2, Bundles purchased : 7 Aggregate diversity: 8 Score: 1.500000, Average bundle size: 2.000000
Rank of user 4 : 0, Size of bundle : 2, Bundles purchased : 8 Aggregate diversity: 10 Score: 1.400000, Average bundle size: 2.000000
Rank of user 5 : 1, Size of bundle : 2, Bundles purchased : 5 Aggregate diversity: 12 Score: 1.500000, Average bundle size: 2.000000
Rank of user 6 : 0, Size of bundle : 2, Bundles purchased : 4 Aggregate diversity: 14 Score: 1.428571, Average bundle size: 2.000000
Rank of user 7 : 1, Size of bundle : 2, Bundles purchased : 3 Aggregate 